# Prerequisite
You need to have a working local installation of Ansys

## Sections
### I. Transmon only
1. Prepare the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on single mode. <br>
1. Get qubit freq and anharmonicity. <br>
1. Calculate EPR of substrate.  <br>
1. (Extra: Calculate surface EPR.) <br>

### II. Resonator only
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Calculate EPR of substrate. <br>
    
### III. Transmon & resonator
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

### IV. Analyze a coupled 2 transmon system. 
1. Finite Element Eigenmode Analysis
1. Identify the mode you want. The mode can inclusively be from 1 to setup.n_modes.
1. Set variables in the Ansys design. As before, we seek 2 modes.
1. Set up the simulation and specify the variables for the sweep.
1. Plot the E-field on the chip's surface.
1. Specify the junctions in the model; in this case there are 2 junctions.
1. Find the electric and magnetic energy stored in the substrate and the system as a whole.
1. Perform EPR analysis for all modes and variations.

In [ ]:
%reload_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

# 1. Analyze the transmon qubit by itself

We will use the analysis package - applicable to most users. Advanced users might want to expand the package, or directly interact with the renderer. The renderer is one of the properties of the analysis class.

### Create the Qbit design

Setup a design of a given dimension. Dimensions will be respected in the design rendering. <br>
Note that the design size extends from the origin into the first quadrant.

In [ ]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

Create a single transmon with one readout resonator and move it to the center of the chip previously defined.

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

gui.rebuild()
gui.autoscale()

### Finite Element Eigenmode Analysis

#### Setup

Select the analysis you intend to run from the `qiskit_metal.analyses` collection.<br>
Select the design to analyze and the tool to use for any external simulation

In [ ]:
from qiskit_metal.analyses.quantization import EigenmodeAndEPR

In [ ]:
eig_qb = EigenmodeAndEPR(design, "hfss")

Review and update the convergence parameters and junction properties by executing following two cells. We exemplify three different methods to update the setup parameters.

In [ ]:
eig_qb.setup

In [ ]:
eig_qb.setup.max_passes = 6   # update single setting
eig_qb.setup.variables['Lj'] = '11 nH'
eig_qb.setup_update(max_delta_f = 0.4, min_freq_ghz = 1.1)   # update multiple settings
eig_qb.setup

#### Execute simulation and verify convergence and EM field
Analyze a single qubit with shorted terminations. Then observe the frequency convergence plot. If not converging, you might want to increase the min_passes value to force the renderer to increase accuracy

In [ ]:
eig_qb.run(name="Qbit", components=['Q1'], open_terminations=[], box_plus_buffer = False)
eig_qb.plot_convergences()

(optional) Captures the renderer GUI

In [ ]:
eig_qb.save_screenshot()

In [ ]:
#
# TODO.....what is this????
#
eig_qb.recompute_convergences("scale_factor='100.2001'")

(optional) Work directly with the convergence numbers

In [ ]:
eig_qb.convergence_f

(optional) You can re-run the analysis after varying the parameters.<br>
Not passing the parameter `components` to the run() method, skips the rendering and tries to run the analysis on the latest design. If a design is not found, the full metal design is rendered.

In [ ]:
eig_qb.setup.min_freq_ghz = 4
eig_qb.run()
eig_qb.convergence_f

Verify that the Electro(magnetic) fields look realistic.

In [ ]:
eig_qb.plot_fields('main')   # TODO:::: Ez, normal component.....decide which field typically on the qbit, or on the crossing between meanders
eig_qb.save_screenshot()

(optional) clear the renderer by removing the fields

In [ ]:
eig_qb.clear_fields()

### EPR Analysis
#### Setup
Identify the non-linear (Josephson) junctions in the model. In this case there's only one, namely 'jj'. Also define the dissipative reference shapes.

In [ ]:
eig_qb.add_junction(name='jj', rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                  Lj_variable='Lj', Cj_variable='Cj')
eig_qb.add_dissipative(category='dielectrics_bulk', name_list=['main'])

Inspect the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
eig_qb.epr_get_stored_energy()

#### Perform EPR for all modes/variations. Then verify spectrum analysis and Hamiltonian

In [ ]:
eig_qb.epr_run_analysis()

Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR.

In [ ]:
eig_qb.epr_spectrum_analysis(cos_trunc = 8, fock_trunc = 7)

Report solved results.

In [ ]:
eig_qb.epr_report_hamiltonian(swp_variable = 'Lj')  # suppose we swept an optimetric analysis vs. inductance Lj_alice

# 2. Analyze the CPW resonator by itself
### Update the design in Metal
Connect the transmon to a CPW. <br>
The other end of the CPW connects to an open to ground termination.

In [ ]:
from qiskit_metal.qlibrary.connectors.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
RouteMeander(design, 'readout',  Dict(
        total_length='6 mm',
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

### Finite Element Eigenmode Analysis

#### Setup

Create a separate analysis object, dedicated to the readout. This allows to retain the Qubit session active, in case we will later need to tweak the design and repeat the simulation. When different renderers are available you could even consider using different more appopriate ones for each simulation steps of this notebook, but for now we will be using the same one.

In [ ]:
eig_rd = EigenmodeAndEPR(design, "hfss")

For the resonator analysis we will use the default setup. Youn can feel free to edit it the same way we did in section 1.

#### Execute simulation and verify convergence and EM field
Analyze the readout in isolation. Select the readout and terminate it with an open on both ends. Note that we are selecting for this analysis both the `readout` component and the `open_to_ground` component. The `open_to_ground` compoent might feel redundant because we are specifying in that open in the `open_terminations`, and the end converging reult is indeed the same. however the `open_to_ground` appears to help the system to ceonverge faster, so we keep it in there.

In [ ]:
eig_rd.run(name="Readout",
           components=['readout', 'open_to_ground'],
           open_terminations=[('readout', 'start'), ('readout', 'end')])
eig_rd.plot_convergences()

In [ ]:
eig_rd.save_screenshot()  # optional

At this time, the renderer maintains the junctions and dissipative definitions set during previous analysis, which in our case it is what we did in section 1 of this notebook. Therefore, even if we do not set them for this analysis, it will reuse the previous information. Consequently, if you already executed section 1, you can skip the execution of the following cell.

In [ ]:
eig_rd.add_dissipative(category='dielectrics_bulk', name_list=['main'])  # Define the reference plane

Recover eigenmode frequencies for each variation.

In [ ]:
eig_rd.epr_get_frequencies()

Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
eig_rd.plot_fields('main')
eig_rd.save_screenshot()

#### Refine

If convergence is not complete, or the EM field is unclear, update the number of passes and re-run the flow (below repeated for convenience)

In [ ]:
eig_rd.setup.max_passes = 15   # update single setting
eig_rd.run()
eig_rd.plot_convergences()

Display the Ansys modeler window again and plot the E-field on the chip's surface with this updated number of passes. <br>
Note that the bright areas have become much smoother compared to the previous image, indicating better convergence.

In [ ]:
eig_rd.plot_fields('main')
eig_rd.save_screenshot()

### EPR Analysis
Find the electric and magnetic energy stored in the readout system.

In [ ]:
eig_rd.epr_get_stored_energy()

# 3. Analyze the combined transmon + CPW resonator system

### Finite Element Eigenmode Analysis

#### Setup

Create a separate analysis object for the combined qbit+readout.

In [ ]:
eig_qres = EigenmodeAndEPR(design, "hfss")

For the resonator analysis we look for 2 eigenmodes - one with stronger fields near the transmon, the other with stronger fields near the resonator. Therefore let's update the setup accordingly.

In [ ]:
eig_qres.setup.n_modes = 2
eig_qres.setup

#### Execute simulation and verify convergence and EM field
Analyze the qubit+readout. Select the qubit and the readout, then finalize with open termination on the other pins.

In [ ]:
eig_qres.run(name="TransmonResonator",
             components=['Q1', 'readout', 'open_to_ground'],
             open_terminations=[('readout', 'end')])
eig_qres.plot_convergences()

In [ ]:
eig_qres.save_screenshot()  # optional

Display the Ansys modeler window again and plot the E-field on the chip's surface. you can select which of the two modes to visualize.

In [ ]:
eig_qres.plot_fields('main', eigenmode=2)
eig_qres.save_screenshot()

### EPR Analysis
At this time, the renderer mantains the junctions and dissipative definitions set during previous analysis, which in our case it is what we did in section 1 of this notebook. Therefore, even if we do not set them for this analysis, it will reuse the previous information. Consequently, if you already executed section 1, you can skip the execution of the following cell.

In [ ]:
eig_qres.add_junction(name='jj', rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                  Lj_variable='Lj', Cj_variable='Cj')
eig_qres.add_dissipative(category='dielectrics_bulk', name_list=['main'])

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
eig_qres.epr_get_stored_energy()

In [ ]:
##### TODO: what is this line for? --> eprd.set_mode(0)

#### Perform EPR for all modes/variations. Then verify spectrum analysis and Hamiltonian

In [ ]:
eig_qres.epr_run_analysis()

Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR.

In [ ]:
eig_qres.epr_spectrum_analysis(cos_trunc = 8, fock_trunc = 7)

Report solved results.

In [ ]:
eig_qres.epr_report_hamiltonian(swp_variable = 'Lj')  # suppose we swept an optimetric analysis vs. inductance Lj_alice

Once you are sure you are done with the qubit analysis, please explicitly release the Ansys session to allow for a smooth close of the external tool.

In [ ]:
eig_qb.close()

In [ ]:
eig_rd.close()

In [ ]:
eig_qres.close()

# 4. Analyze a coupled 2-transmon system
### Create the design

This is a different system than the one analyzed in sections 1,2,3. Therefore, let's start by deleting the design currntly in the Qiskit Metal GUI (if any).

In [ ]:
design.delete_all_components()

Next, we create the `TwoTransmon` design, consisting of 2 transmons connected by a short coupler.

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.interconnects.straight_path import RouteStraight

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x = '1.0 mm',
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=-1,loc_H=+1, pad_width='200um')
    )))

coupler = RouteStraight(design, 'coupler', Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='Q2', pin='readout')), ))

gui.rebuild()
gui.autoscale()

Let's observe the current table describing the junctions in the qiskit metal design

In [ ]:
design.qgeometry.tables['junction']

You can observe in the table above that every junction has been assigned a default inductance, capacitance and resistance values, based on the originating component class `default_options`. In this example we intend to replace those values with a variable name, which will later be set directly in the renderer. Therefore, let's proceed with updating these values in the qubit instances, and then propagate the update to the table with a `rebuild()`.
After executing the cell below, you can observe the change by re-executing the cell above.

In [ ]:
# TODO: fold this inside either an analysis class method, or inside the analysis class setup

qcomps = design.components  # short handle (alias)
qcomps['Q1'].options['hfss_inductance'] = 'Lj1'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj1'
qcomps['Q2'].options['hfss_inductance'] = 'Lj2'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj2'
gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table

### Finite Element Eigenmode Analysis

#### Setup

Let's start the analysis by creating the appropriate analysis object.

In [ ]:
eig_2qb = EigenmodeAndEPR(design, "hfss")

Now let us update the setup of this analysis to reflect what we plan to do:
* define the variables that we have assigned to the inductance and capacitance of the junctions;
* increase accuracy of the convergence;
* observe the eigenmode corresponding to both qubits.

In [ ]:
eig_2qb.setup.max_passes = 15
eig_2qb.setup.max_delta_f = 0.05
eig_2qb.setup.n_modes = 2
eig_2qb.setup.variables = {'Lj1': '13 nH', 'Cj1': '0 fF',
                           'Lj2': '9 nH', 'Cj2': '0 fF'}
eig_2qb.setup

By default, the analysis will be done on all components that we will list in the `run()` method, but the analysis needs to know how much of the ground plane around the qubit to consider. One could use the declared chip dimension by passing the parameter `bux_plus_buffer = False` to the run() method. However, its default (when said parameter is omitted) is to consider the ground plane to be as big as the minimum enclosing rectangle plus a set buffer. The default buffer value is `200um`, while in the cell below we will increase as an example that buffer to `500um`.

In [ ]:
# TODO: fold this inside either an analysis class method, or inside the analysis class setup

eig_2qb.renderer.options['x_buffer_width_mm'] = 0.5
eig_2qb.renderer.options['y_buffer_width_mm'] = 0.5
eig_2qb.renderer.options

Let's finally run the cap extraction simulation and observe the convergence.

In [ ]:
eig_2qb.run(name="TwoTransmons",
            components=['coupler', 'Q1', 'Q2'])
eig_2qb.plot_convergences()

In [ ]:
eig_2qb.save_screenshot()  # optional

Display the Ansys modeler window again and plot the E-field on the chip's surface. Since we have analyzed 2 modes, you will need to select which mode to visualize. The default is mode 1, but the mode can inclusively be any integer between 1 and `setup.n_modes`.

In [ ]:
eig_2qb.plot_fields('main', eigenmode=2)
eig_2qb.save_screenshot()

### EPR Analysis
#### Setup
Identify the non-linear (Josephson) junctions in the model. in this case there are 2 junctions, which we will refer to as `jj1` and `jj2`. Also define the dissipative reference shapes.

In [ ]:
eig_2qb.add_junction(name='jj1', rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                  Lj_variable='Lj1', Cj_variable='Cj1')
eig_2qb.add_junction(name='jj2', rect='JJ_rect_Lj_Q2_rect_jj', line='JJ_Lj_Q2_rect_jj_',
                  Lj_variable='Lj2', Cj_variable='Cj2')
eig_2qb.add_dissipative(category='dielectrics_bulk', name_list=['main'])

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
eig_2qb.epr_get_stored_energy()

#### Perform EPR for all modes/variations. Then verify spectrum analysis and Hamiltonian

In [ ]:
eig_2qb.epr_run_analysis()

Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR.

In [ ]:
eig_2qb.epr_spectrum_analysis(cos_trunc = 8, fock_trunc = 7)

Report solved results.

In [ ]:
eig_2qb.epr_report_hamiltonian(swp_variable = 'Lj1')  # suppose we swept an optimetric analysis vs. inductance Lj1_alice

In [ ]:
# TODO: should we develop this further?

# for lj in range(12,16, 1):
#    pinfo.design.set_variable('Lj1', f'{lj} nH')
#    pinfo.setup.analyze()

Release Ansys's session

In [ ]:
eig_2qb.close()

In [ ]:
# gui.main_window.close()